## ROZWIĄZYWANIE UKŁADÓW RÓWNAŃ LINIOWYCH 

### $A \vec{x} = \vec{b}, \,$  gdzie: $A$ - macierz

In [1]:
import numpy as np
import scipy.linalg

### 1) metoda eliminacji Gaussa z częściowym wyborem elementu podstawowego

Szukamy takiego elementu $|a_{i_k k}^{(k)}|$, żeby:
$$
|a_{i_k k}^{(k)}| = \max\limits_{k \leq i \leq n} |a_{ik}^{(k)}|
$$


Następnie w macierzy $A^{(k)}$ zamieniamy wiersze o numerach $i_k$ oraz $k$, a w wektorze $b^{(k)}$ składowe o takich samych numerach. 

In [2]:
def swapping(M: np.ndarray, bb: np.ndarray) -> tuple[np.ndarray, np.ndarray, int]:
    matrix = M.copy()
    b = bb.copy()
    n = len(matrix)
    count_swapped = 0
    for i in range(n):
        max_el = np.abs(matrix[i, i])
        for m in range(i+1, n):
            if max_el < np.abs(matrix[m, i]):
                max_el = np.abs(matrix[m, i])
                matrix[[i, m]] = matrix[[m, i]]
                b[[i, m]] = b[[m, i]]
                count_swapped +=1 

        for j in range(i+1, n):
            k = matrix[j, i] / matrix[i, i]
            matrix[j] = matrix[j] - k * matrix[i]
            b[j] = b[j] - k * b[i]

    return matrix, b, count_swapped

In [3]:
def solve(M: np.ndarray, bb: np.ndarray) -> np.ndarray:
    matrix = M.copy()
    b = bb.copy()
    final_matrix, final_b, _ = swapping(matrix, b)
    n = len(final_matrix)
    x = np.zeros_like(final_b)

    for i in range(n-1, -1, -1):
        x[i] = final_b[i]
        for j in range(i + 1, n):
            x[i] = x[i] - final_matrix[i][j] * x[j]
  
        x[i] = x[i] / final_matrix[i][i] 

    return x

### 2) metoda Gaussa-Seidla
$$x^{n+1} = D^{-1}b - D^{-1}L x^{n+1} - D^{-1}Ux^{n}$$

gdzie: $D$ - nieosobliwa macierz diagonalna, $U$ - macierz górnotrójkątna, $L$ - macierz dolnotrójkątna

In [4]:
def GS(matrix: np.ndarray, b: np.ndarray, iter_num: int = 1000, tol: float = 10**(-9)):
    n = len(b)
    x = np.zeros_like(b, dtype=np.float64) 

    D = np.diag(np.diag(matrix)) 
    L = np.tril(matrix, k=-1)   
    U = np.triu(matrix, k=1)    

    D_inv = np.diag(1 / np.diag(D))
    T = -D_inv @ (L + U)
    spec = np.max(np.abs(np.linalg.eigvals(T)))

    if spec >= 1:
        raise ValueError("Brak zbieżności - promień spektralny ≥ 1")

    for _ in range(iter_num):
        x_old = x.copy()

        for i in range(n):
            sum_L = np.dot(L[i, :], x) 
            sum_U = np.dot(U[i, :], x_old)
            x[i] = D_inv[i, i] * (b[i] - sum_L - sum_U)

        if np.linalg.norm(x - x_old, ord=np.inf) < tol:
            return x

    return x

### 3) wyznacznik macierzy

In [5]:
def det_matrix(M: np.ndarray) -> float:
    matrix = M.copy()
    n = len(matrix)
    swapped_matrix, _, det_count = swapping(matrix, np.eye(n))
    det = 1
    for i in range(n):
        det *= swapped_matrix[i][i]
    det *= (-1)** det_count
    return det

### 4) macierz odwrotna

In [6]:
def inverse(M: np.ndarray) -> np.ndarray:
    matrix = M.copy()
    n = len(matrix)
    swapped_matrix, inverse_matrix, _ = swapping(matrix, np.eye(n))
    detM = det_matrix(matrix)
    if detM == 0:
        raise ValueError('macierz osobliwa -> macierz odwrotna nie istnieje !')
    for i in range(n-1, -1, -1):
        for j in range(i-1, -1, -1):
            k = swapped_matrix[j][i]/ swapped_matrix[i][i]
            swapped_matrix[j] = swapped_matrix[j] - k*swapped_matrix[i]
            inverse_matrix[j] = inverse_matrix[j] - k*inverse_matrix[i]

        p = swapped_matrix[i,i]
        swapped_matrix[i] = swapped_matrix[i]/p
        inverse_matrix[i] = inverse_matrix[i]/p

    return inverse_matrix